In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/turkish-sales-comments/hepsiburada.csv


In [2]:
import numpy as np
import pandas as pd
#pandas data setı okumaya ve duzenlemeye yardımcı olur
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding, CuDNNGRU
#from keras.models import Sequential
from keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [3]:
dataset = pd.read_csv('../input/turkish-sales-comments/hepsiburada.csv')
dataset.head()
#dataframe olrak yuklendı

,Rating,Review
0,1,3 yıldır tık demedi. :)
1,1,3 yıldır kullanıyorum müthiş
2,1,Ürün bugün elime geçti çok fazla inceleme fırs...
3,1,Almaya karar verdim. Hemencecik geldi. Keyifle...
4,1,Günlük kullanımınızı çok çok iyi karsılıyor kı...


In [4]:
target = dataset['Rating'].values.tolist()
data = dataset['Review'].values.tolist()
cutoff = int(len(data) * 0.80)
x_train, x_test = data[:cutoff], data[cutoff:]
y_train, y_test = target[:cutoff], target[cutoff:]
print(x_train[100])
print(y_train[100])

Bu fiyata bu kalite kaçırmayın derim 
1


In [5]:
num_words = 10000
tokenizer = Tokenizer(num_words=num_words)
#en sık gecen 10bın kelıme kelıme haznesınde
tokenizer.fit_on_texts(data)
#tokenlestırme yapıldı
#tokenizer.word_index  # her kelimeye bir sayı karşılık geldi

In [6]:
x_train_tokens = tokenizer.texts_to_sequences(x_train) # x_trainde yorumlarım kelime şeklinde bunları tokenlaştırarak int atıyorum her kelimeye
#data ıcın yapılan durumun aynısı eğitim verisi içinde yapılıyor
print(x_train[800])
#secılen cumle asagıda
print(x_train_tokens[800])
#secılen cumlenın token halı asagıda

ürünü alalı 3 hafta kadar oldu. aralıksız kullanıyorum bilgisyarım sürekli açık durur ve ben günde yaklaşık 12 saat başındayım mousesu çok kullanırım. şimdiye kadar bir problem yaşamadım ve çok memnunum almak isteyenlere tavsiye ederim.
[19, 341, 52, 194, 30, 54, 7992, 55, 209, 603, 7887, 3, 36, 114, 164, 479, 85, 1, 1682, 782, 30, 2, 380, 326, 3, 1, 81, 132, 562, 9, 10]


In [7]:
x_test_tokens = tokenizer.texts_to_sequences(x_test)
#test ıcındekı cumleler tokenlestırılıyor


# bizin imputumuz farklı kelimeelrden oluşuyor bir yorum 50 kelimeyken bir yorum 10 kelime bunu normalize etmemiz gerekiyor.
# tüm yorumları rnn e sokmak için bir standart belirliyoruz.
# eğer yorum belirlediğimiz sınırdan büyükse sileceğiz küçükse 0 ile dolduracağız.
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]


num_tokens = np.array(num_tokens)
# rnn belırlı boyutta verı verılmelı bu yuzden tum verılerı aynı boyuta getırmelıyız
# np array amac daha kolay ıslem yapmak
np.mean(num_tokens) # # yorumlarda ortalama olarak 20.74 tane token bulunuyor, kelime yani 
#ortalama 20 token var her cumlede

20.744703220162876

In [8]:
print(np.max(num_tokens))
# en con 195 kelıme
print(np.argmax(num_tokens))
#en uzun olan 295 sırasını bulmak ıcın
print(x_train[21941])
#en uzuna bakalım

295
21941
Özellikle bu kısma yazıyorum iyice okuyunuz,cihazın hızı çok iyi.Isınma normal boyutlarda.Kamerası elinizi sabit tutarsanız ve gündüz çok net gece ise çok net çekmez.Görüntülü konuşma özelliği yok ancak uygulama ile olur,anten çekimi gayet iyi.Parmak izi okuyucusu gayet hızlı bazen tuşu silmenizi istiyor ve okuyamıyor kirden vs...Ön kamera da tatmin edici,çekim esnasında ekran beyaz ışık vererek flaş görevi görür.Batarya bana 1 hafta gidiyor sık kullanımda ise 2-3 gün gidiyor.Hızlı şarj 1 saatte doluyor , şekilleri ele oturuyor ve şık bir görüntü var.Telefonu aldığım gün gittim ve ilk girdiğim yerden ekran koruyucu ve kılıf buldum.Kulaklık sesi çok net ve yüksek ancak kendi hoparlörü biraz zayıf sesi.Ekrana bakarken açık kalma özelliği yok.Diğer akıllı cihaz özellikleri %90 ı bu cihazda mevcut.Güç tasarrufu 2 ayrı modu var ve çok başarılı çalışıyor.4gb ram var genelde yarısı boş kalıyor.Bir de yeni cihazların çoğu titreşimi az ve sesi de az çıkıyor.Bu cihaz da içine dahil...B

In [9]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
print(max_tokens) #burada mean ve standart sapma degerlerıyle kac kelıme alsak daha ıyı olur onu secıyoruz
print(np.sum(num_tokens < max_tokens) / len(num_tokens))
#kac tane cumle 59 kaplıyor yaklasık yuzde 95 
# # %95 inde 59 dan az token varmış %5 te ise 59 dan fazla token varmış

59
0.9597982726686571


In [10]:
#padding işlemi yapıyoruz. padding ile 59 tokenı sabit hale getiriyoruz.
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens)
#traın ıcın 59 dan sonrası sılıyorz
print(x_train_pad.shape)
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens)
#test ıcın 59 dan sonrası sılıyorz
print(x_test_pad.shape)

(194797, 59)
(48700, 59)


In [11]:
print(np.array(x_train_tokens[800]))
print(x_train_pad[800])
# burada paddıng eklenmısler var

[  19  341   52  194   30   54 7992   55  209  603 7887    3   36  114
  164  479   85    1 1682  782   30    2  380  326    3    1   81  132
  562    9   10]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
   19  341   52  194   30   54 7992   55  209  603 7887    3   36  114
  164  479   85    1 1682  782   30    2  380  326    3    1   81  132
  562    9   10]


In [12]:
idx = tokenizer.word_index
#kelımeler ve buna karsılık gelen degerler burada
inverse_map = dict(zip(idx.values(), idx.keys())) # sayıyı verdiğimizde o sayıya karşılık gelen kelimeyi bulabiliriz.

#bu işlem ile hangı kelıme hangı token aldı bunu hesaplıyoruz
# cunku keras kelıme token yapıyo ama gerı cozemıyoruz bu yuzden onemlı

def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token!=0]
    text = ' '.join(words)
    return text
#buraya hangı token versek text halınde gerı donucek

"""
def tokens_to_string(tokens):
    words=list()
    for token in tokens:
        if token!=0:
            words.append(inverse_map[token])
        #words = [inverse_map[token] for token in tokens if token!=0]
        text = ' '.join(words)
    return text
"""

"\ndef tokens_to_string(tokens):\n    words=list()\n    for token in tokens:\n        if token!=0:\n            words.append(inverse_map[token])\n        #words = [inverse_map[token] for token in tokens if token!=0]\n        text = ' '.join(words)\n    return text\n"

In [13]:
print(x_train[800])
#bu orjınal 800 ındextekı cumle
print(tokens_to_string(x_train_tokens[800]))  # bazı kelimeler eksik bunun sebebi tokenlaştırma yaparken 10k olarak sınırlamıştık

ürünü alalı 3 hafta kadar oldu. aralıksız kullanıyorum bilgisyarım sürekli açık durur ve ben günde yaklaşık 12 saat başındayım mousesu çok kullanırım. şimdiye kadar bir problem yaşamadım ve çok memnunum almak isteyenlere tavsiye ederim.
ürünü alalı 3 hafta kadar oldu aralıksız kullanıyorum sürekli açık durur ve ben günde yaklaşık 12 saat çok kullanırım şimdiye kadar bir problem yaşamadım ve çok memnunum almak isteyenlere tavsiye ederim


* model eğitiminde ;
* öncelikle embedded matrisi oluşturup 3 katmanlı GRU ile eğitim yapacağız
* GRU sonucunda tek bir değer oluşacak bu değer 1 e yada 0 a yakınsa ona göre sonuca varacağız.

In [14]:
model = Sequential()
embedding_size = 50
#her keılmeye karsılık gelen 50 uzunlugunda vektor var kelıme degerlının tutuldugu
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='embedding_layer'))
#ınput belırlenen maksımum deger kadar
#cıkıs 50 vektor 
#gırıs ıse verıler kelımeler (10bın uzunlugunda verı olucak)
model.add(GRU(units=16, return_sequences=True))
#return true olma sebebı tum cıkısların dıger aglara aktarılması, eger false olsaysı sadece son output cıkcaktı
#eger gpu olurda CuDNNGRU kullanılabılır, sadece nvdıa nın kartında calısyo ve sadece hız farkı var
model.add(GRU(units=8, return_sequences=True))
model.add(GRU(units=4)) #default false o yuzden yazmaya gerek yok
model.add(Dense(1, activation='sigmoid'))
optimizer = Adam(lr=1e-3)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
model.summary()

# sinir ağımıza input olarak bir yorum veriyorum. bu yorum tokenlardan oluşuypr.
#yorumun içinde her kelimeye karşılık gelen bir kelime vektörü var
#embedding layerda yoruma karşılık gelen vektörler bulunuyor. bu vektörler GRU layerına output olarak veriliyor.
# en son olarak 3 farklı gru layerında işlem gördükten sonra output layerına gönderiliyor.

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 59, 50)            500000    
_________________________________________________________________
gru (GRU)                    (None, 59, 16)            3216      
_________________________________________________________________
gru_1 (GRU)                  (None, 59, 8)             600       
_________________________________________________________________
gru_2 (GRU)                  (None, 4)                 156       
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 503,977
Trainable params: 503,977
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(np.array(x_train_pad), np.array(y_train), epochs=5, batch_size=256)


Epoch 1/5
761/761 [==============================] - 451s 584ms/step - loss: 0.2304 - accuracy: 0.9456
Epoch 2/5
761/761 [==============================] - 445s 584ms/step - loss: 0.0898 - accuracy: 0.9712
Epoch 3/5
761/761 [==============================] - 443s 582ms/step - loss: 0.0704 - accuracy: 0.9782
Epoch 4/5
761/761 [==============================] - 442s 581ms/step - loss: 0.0526 - accuracy: 0.9852
Epoch 5/5
761/761 [==============================] - 443s 582ms/step - loss: 0.0416 - accuracy: 0.9890


In [16]:
result = model.evaluate(np.array(x_test_pad), np.array(y_test))

1522/1522 [==============================] - 64s 42ms/step - loss: 0.1583 - accuracy: 0.9532


In [17]:
print(result[1]) # accuracy

0.9532443284988403


In [18]:
y_pred = model.predict(x=x_test_pad[0:1000]) # test setindeki ilk 1000 yorum # normalde sütun halde 
y_pred = y_pred.T[0] #transpozunu alarak yatay hale getiriryoruz.


In [19]:
cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in y_pred]) # 0.5den büyük olanları 1 kabul ettik
cls_true = np.array(y_test[0:1000])  # burdada 0dan 1000 e kadar olan yorumların etiketlerini aldık
incorrect = np.where(cls_pred != cls_true)
incorrect = incorrect[0]

In [20]:
len(incorrect)

62

In [21]:
idx = incorrect[1]
idx

31

In [22]:
text = x_test[idx]
print(text)

Hastalıktan yeni yeni kendine gelen asil saf kan kangalım kendine gelir gelmez önceki yeminden değil bu yemden yemeye başladı..


In [23]:
y_pred[idx]


0.29061309

In [24]:
cls_true[idx]

1

In [25]:
text1 = "bu ürün çok iyi herkese tavsiye ederim"
text2 = "kargo çok hızlı aynı gün elime geçti"
text3 = "büyük bir hayal kırıklığı yaşadım bu ürün bu markaya yakışmamış"
text4 = "mükemmel"
text5 = "tasarımı harika ancak kargo çok geç geldi ve ürün açılmıştı tavsiye etmem"
text6 = "hiç resimde gösterildiği gibi değil"
text7 = "kötü yorumlar gözümü korkutmuştu ancak hiçbir sorun yaşamadım teşekkürler"
text8 = "hiç bu kadar kötü bir satıcıya denk gelmemiştim ürünü geri iade ediyorum"
text9 = "tam bir fiyat performans ürünü"
text10 = "beklediğim gibi çıkmadı"
texts = [text1, text2, text3, text4, text5, text6, text7, text8, text9, text10]

In [26]:
tokens = tokenizer.texts_to_sequences(texts) # önce tokenlaştırdık
tokens

[[5, 4, 1, 6, 70, 9, 10],
 [27, 1, 15, 94, 40, 29, 150],
 [104, 2, 1032, 2333, 1466, 5, 4, 5, 1779],
 [53],
 [553, 61, 82, 27, 1, 458, 33, 3, 4, 9, 1031],
 [46, 1096, 6419, 20, 50],
 [177, 735, 7728, 82, 263, 105, 326, 16],
 [46, 5, 30, 177, 2, 1717, 1244, 19, 591, 677, 83],
 [74, 2, 28, 111, 19],
 [637, 20, 857]]

In [27]:
tokens_pad = pad_sequences(tokens, maxlen=max_tokens) # sinir ağına vereceğimiz inputun boyutunu 59 yaptık bunuda 59 yaptım.
print(tokens_pad.shape)


(10, 59)


In [28]:
text_pred=model.predict(tokens_pad)
text_pred


array([[0.9976432 ],
       [0.98611313],
       [0.02133275],
       [0.996267  ],
       [0.02345196],
       [0.04239832],
       [0.9971186 ],
       [0.01039133],
       [0.9959253 ],
       [0.0497527 ]], dtype=float32)

In [29]:
text_pred = np.array([1.0 if p>0.5 else 0.0 for p in text_pred])
print(text_pred)

[1. 1. 0. 1. 0. 0. 1. 0. 1. 0.]
